In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import collections

In [53]:
results_path = "./results_3_2_3_fe4+xgb2"
if not os.path.exists(results_path):
    os.makedirs(results_path)

In [54]:
attributes = pd.read_pickle("../attributes/attr.pkl")
attributes["session_id"] = attributes["session_id"].astype('int64')
attributes["user_id_hash"] = attributes["user_id_hash"].astype('object')


get mean values. Choose columns by the number of missing values and from the feature importsnce from xgboost models

In [55]:
all_cols = ["user_id_hash","attr_0","attr_1","attr_2","attr_3","attr_5"
               ,"attr_7","attr_8","attr_10","attr_13","attr_14","attr_15","attr_16","attr_17"
               ,"attr_18","attr_19","attr_25","attr_26","attr_27","attr_28"]

df_attr = attributes[all_cols].groupby(["user_id_hash"]).mean().reset_index()

In [59]:
useful_cols = ["user_id_hash","attr_0","attr_1","attr_2"
               ,"attr_8","attr_10","attr_13","attr_14","attr_16","attr_17"]

df_attr = df_attr[useful_cols].fillna(0)

# Load Data

In [60]:
events_path = "../events/events.csv"
sessions_path = "../sessions/sessions.csv"

In [61]:
columns = ["session_id", "event", "event_timestamp", "event_value", "user_id_hash"]
events = pd.read_csv(events_path, usecols=columns)

columns = ["session_id", "start_timestamp", "timezone_offset", "previous_sessions_duration", 
           "user_created_timestamp", "is_user_first_session", "is_session", "is_developer", 
          "is_wau", "is_mau", "session_index", "user_id_hash"]
sessions = pd.read_csv(sessions_path, usecols=columns)

## Convert timestamp

In [62]:
events["timestamp"] = pd.to_datetime(events["event_timestamp"], unit='ms')
sessions["uc_timestamp"] = pd.to_datetime(sessions["user_created_timestamp"], unit='ms')
sessions["st_timestamp"] = pd.to_datetime(sessions["start_timestamp"], unit='ms')

# Label and Feature Functions

In [63]:
def create_labels(events, sessions, start_window, end_window):
    events_8 = events[(events["timestamp"]>= start_window) & (events["timestamp"]<= end_window) & 
                      (events["event"]=='8')]
    tempdf = events_8.groupby(["user_id_hash"]).agg({"event":"count"}).reset_index()
    tempdf["target"] = tempdf["event"].apply(lambda x: 0 if x==0 else 1)

    df_labels = pd.DataFrame({"user_id_hash":sessions.user_id_hash.unique()})
    df_labels = pd.merge(df_labels, tempdf[["user_id_hash", "target"]], on="user_id_hash", how="left")
    df_labels["target"] = df_labels["target"].apply(lambda x: 0 if pd.isnull(x) else 1)
    return df_labels

### feature: days_since_uc, days_used
Days since user creation, distinct says used

In [64]:
def used_days_feature(sessions, used_from_date):
    df_used_days = sessions[["user_id_hash", "uc_timestamp", "st_timestamp"]] \
                            .groupby(["user_id_hash"]) \
                            .agg({"uc_timestamp":"min",
                                  "st_timestamp":lambda x: x.dt.date.nunique()}).reset_index()

    df_used_days["uc_timestamp"] = (pd.to_datetime(used_from_date) - df_used_days["uc_timestamp"]).dt.days + 1
    df_used_days.columns = ["user_id_hash", "days_since_uc", "days_used"]
    
    return df_used_days

### feature: session_num, session_num_per_day, session_duration_per_session , last_session_days

In [65]:
def sessions_features(sessions, sess_from_date):
    df_session_num = pd.DataFrame({"user_id_hash":sessions.user_id_hash.unique()})

    tempdf = sessions[(sessions["is_session"]==True) 
                       & (sessions["is_developer"]==False) 
                       & (sessions["st_timestamp"]<sess_from_date)]
    tempdf = tempdf[["session_id", "user_id_hash", "previous_sessions_duration", "st_timestamp"]] \
                       .groupby(["user_id_hash"]) \
                       .agg({"session_id":"count", 
                             "previous_sessions_duration":"mean",
                             "st_timestamp": lambda x: (pd.to_datetime(sess_from_date) - x.max()).days}) \
                       .reset_index()

    df_session_num = pd.merge(df_session_num, tempdf, on="user_id_hash", how="left")
    df_session_num.fillna(0, inplace=True)
    df_session_num.columns = ["user_id_hash", "session_num", "session_duration_per_session", "last_session_days"]
    
    return df_session_num

### feature: purchases_num, purchases_num_per_day, whether_purch

In [66]:
def purchase_features(events, purch_from_date):
    events8_feat = events[(events["timestamp"]<purch_from_date) & (events["event"]=='8')]
    tempdf = events8_feat.groupby(["user_id_hash"]).agg({"event":"count"}).reset_index()

    df_purch = pd.DataFrame({"user_id_hash":sessions.user_id_hash.unique()})
    df_purch = pd.merge(df_purch, tempdf, how="left")
    df_purch.fillna(0, inplace=True)
    df_purch.columns = ["user_id_hash", "purchases_num"]
    df_purch["whether_purch"] = df_purch["purchases_num"].apply(lambda x: 1 if x>0 else 0)

    return df_purch

### feature: last7d_purch, last14d_purch
Number of purchases in previous 7 or 14 days

In [67]:
def recent_purchases_feature(events, purch_from_date, num_days):
    events8_feat = events[(events["timestamp"]<purch_from_date) & (events["event"]=='8')]
    recent_purch_filter = events8_feat.timestamp >= np.datetime64(purch_from_date) - np.timedelta64(num_days, 'D')

    tempdf = events8_feat[recent_purch_filter].groupby(["user_id_hash"]).agg({"event":"count"}).reset_index()
    recent_purch = pd.DataFrame({"user_id_hash":sessions.user_id_hash.unique()})

    recent_purch = pd.merge(recent_purch, tempdf, how="left")
    recent_purch.fillna(0, inplace=True)

    col_name = f"last{num_days}d_purch"
    recent_purch.columns = ["user_id_hash", col_name]
    
    return recent_purch


### feature: events_counts

In [68]:
def events_count_encoding(events, purch_from_date):
    event_list = ['45']
    recent_events_filter = events.timestamp >= np.datetime64(purch_from_date) - np.timedelta64(14, 'D')
    
    events_subset = events[(recent_events_filter) & (events.event.isin(event_list))]
    tempdf = events_subset.groupby(["user_id_hash", "event"]).count().reset_index()
    tempdf = tempdf.pivot_table('session_id', 'user_id_hash', columns='event').reset_index()

    df_event_counts = pd.DataFrame({"user_id_hash":sessions.user_id_hash.unique()})
    df_event_counts = pd.merge(df_event_counts, tempdf, how="left").fillna(0)
    
#     temp_df = pd.DataFrame({"user_id_hash":sessions.user_id_hash.unique()})
#     events_piv = events[events["timestamp"] <= purch_from_date].groupby(['user_id_hash', 'event']).size().reset_index()
#     events_piv = events_piv.pivot_table(0, 'user_id_hash', columns='event').reset_index()
#     events_counts = pd.merge(temp_df, events_piv, how='left', on='user_id_hash').fillna(0)
    
    return df_event_counts

# Combine features and labels

In [69]:
def create_features(events, sessions, from_date, to_date):
    df_used_days = used_days_feature(sessions, from_date)
    df_session_num = sessions_features(sessions, from_date)
    df_purch = purchase_features(events, from_date)
    
    recent_purch_days = pd.to_datetime(to_date) - pd.to_datetime(from_date)
    df_recent_purch = recent_purchases_feature(events, from_date, recent_purch_days.days+1)
#     df_event_counts = events_count_encoding(events, from_date)

    df = pd.merge(df_used_days, df_session_num)
    df = pd.merge(df, df_purch)
    df = pd.merge(df, df_purch)
    df = pd.merge(df, df_recent_purch)
#     df = pd.merge(df, df_event_counts)
    
    df["purchases_num_per_day"] = df["purchases_num"]/df["days_used"]
    df["session_num_per_day"] = df["session_num"]/df["days_used"]
    
    return df

# 7 Day Training dataset

In [71]:
df7t_part = create_features(events, sessions, '2018-12-08', '2018-12-14')
df_labels = create_labels(events, sessions, '2018-12-08', '2018-12-14')

In [72]:
df7t = pd.merge(df7t_part, df_labels)
df7t = pd.merge(df7t, df_attr, on="user_id_hash", how="left")
df7t = df7t.fillna(0)

In [73]:
df7t.shape

(621106, 21)

In [74]:
df7t.head()

,user_id_hash,days_since_uc,days_used,session_num,session_duration_per_session,last_session_days,purchases_num,whether_purch,last7d_purch,purchases_num_per_day,...,target,attr_0,attr_1,attr_2,attr_8,attr_10,attr_13,attr_14,attr_16,attr_17
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,28,6,22.0,7.129631e+06,20.0,8.0,1,0.0,1.333333,...,0,2.0,37.78125,15.039062,42.90625,0.0,16.8125,42.375,21.796875,3.000000
1,000059859ec188af6035870faf885c3038cedda05b3a54...,16,4,10.0,4.524304e+06,13.0,1.0,1,0.0,0.250000,...,0,0.0,0.00000,1.000000,6.87500,0.0,0.5000,2.500,9.000000,7.750000
2,000062e9be78f3da274fec338e78f89d12000e781967f2...,62,7,27.0,1.025302e+07,55.0,0.0,0,0.0,0.000000,...,0,0.5,0.00000,0.000000,12.12500,0.0,0.0000,2.500,29.593750,13.335938
3,00006ad44c0d33a2a526caf75514643639b7b7070dfffd...,51,3,7.0,4.104550e+06,49.0,0.0,0,0.0,0.000000,...,0,0.0,0.00000,0.000000,5.25000,0.0,0.0000,3.000,9.000000,9.500000
4,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,16,3,3.0,9.135007e+05,14.0,0.0,0,0.0,0.000000,...,0,0.0,0.00000,0.000000,0.50000,0.0,0.0000,1.000,42.500000,0.500000


In [75]:
df7t.to_csv(results_path + "/traindata_7.csv", index=False)

# 14 Day Training dataset

In [76]:
df14t_part = create_features(events, sessions, '2018-12-01', '2018-12-14')
df_labels = create_labels(events, sessions, '2018-12-01', '2018-12-14')


In [77]:
df14t = pd.merge(df14t_part, df_labels)
df14t = pd.merge(df14t, df_attr, on="user_id_hash", how="left")
df14t = df14t.fillna(0)

In [78]:
df14t.shape

(621106, 21)

In [79]:
df14t.head()

,user_id_hash,days_since_uc,days_used,session_num,session_duration_per_session,last_session_days,purchases_num,whether_purch,last14d_purch,purchases_num_per_day,...,target,attr_0,attr_1,attr_2,attr_8,attr_10,attr_13,attr_14,attr_16,attr_17
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,21,6,22.0,7.129631e+06,13.0,8.0,1,0.0,1.333333,...,0,2.0,37.78125,15.039062,42.90625,0.0,16.8125,42.375,21.796875,3.000000
1,000059859ec188af6035870faf885c3038cedda05b3a54...,9,4,10.0,4.524304e+06,6.0,1.0,1,1.0,0.250000,...,0,0.0,0.00000,1.000000,6.87500,0.0,0.5000,2.500,9.000000,7.750000
2,000062e9be78f3da274fec338e78f89d12000e781967f2...,55,7,27.0,1.025302e+07,48.0,0.0,0,0.0,0.000000,...,0,0.5,0.00000,0.000000,12.12500,0.0,0.0000,2.500,29.593750,13.335938
3,00006ad44c0d33a2a526caf75514643639b7b7070dfffd...,44,3,7.0,4.104550e+06,42.0,0.0,0,0.0,0.000000,...,0,0.0,0.00000,0.000000,5.25000,0.0,0.0000,3.000,9.000000,9.500000
4,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,9,3,3.0,9.135007e+05,7.0,0.0,0,0.0,0.000000,...,0,0.0,0.00000,0.000000,0.50000,0.0,0.0000,1.000,42.500000,0.500000


In [80]:
df14t.to_csv(results_path + "/traindata_14.csv", index=False)

# 7 Day Entire dataset

In [81]:
df7a_part = create_features(events, sessions, '2018-12-14', '2018-12-20')


In [82]:
df7a = pd.merge(df7a_part, df_attr)

In [83]:
df7a.head()

,user_id_hash,days_since_uc,days_used,session_num,session_duration_per_session,last_session_days,purchases_num,whether_purch,last7d_purch,purchases_num_per_day,session_num_per_day,attr_0,attr_1,attr_2,attr_8,attr_10,attr_13,attr_14,attr_16,attr_17
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,34,6,23.0,7.693981e+06,0.0,8.0,1,0.0,1.333333,3.833333,2.0,37.78125,15.039062,42.90625,0.0,16.8125,42.375,21.796875,3.000000
1,000059859ec188af6035870faf885c3038cedda05b3a54...,22,4,10.0,4.524304e+06,19.0,1.0,1,0.0,0.250000,2.500000,0.0,0.00000,1.000000,6.87500,0.0,0.5000,2.500,9.000000,7.750000
2,000062e9be78f3da274fec338e78f89d12000e781967f2...,68,7,27.0,1.025302e+07,61.0,0.0,0,0.0,0.000000,3.857143,0.5,0.00000,0.000000,12.12500,0.0,0.0000,2.500,29.593750,13.335938
3,00006ad44c0d33a2a526caf75514643639b7b7070dfffd...,57,3,7.0,4.104550e+06,55.0,0.0,0,0.0,0.000000,2.333333,0.0,0.00000,0.000000,5.25000,0.0,0.0000,3.000,9.000000,9.500000
4,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,22,3,3.0,9.135007e+05,20.0,0.0,0,0.0,0.000000,1.000000,0.0,0.00000,0.000000,0.50000,0.0,0.0000,1.000,42.500000,0.500000


In [84]:
df7a.to_csv(results_path + "/allfeatures_7.csv", index=False)

# 14 Day Entire dataset

In [85]:
df14a_part = create_features(events, sessions, '2018-12-14', '2018-12-27')


In [86]:
df14a = pd.merge(df14a_part, df_attr)

In [87]:
df14a.head()

,user_id_hash,days_since_uc,days_used,session_num,session_duration_per_session,last_session_days,purchases_num,whether_purch,last14d_purch,purchases_num_per_day,session_num_per_day,attr_0,attr_1,attr_2,attr_8,attr_10,attr_13,attr_14,attr_16,attr_17
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,34,6,23.0,7.693981e+06,0.0,8.0,1,0.0,1.333333,3.833333,2.0,37.78125,15.039062,42.90625,0.0,16.8125,42.375,21.796875,3.000000
1,000059859ec188af6035870faf885c3038cedda05b3a54...,22,4,10.0,4.524304e+06,19.0,1.0,1,0.0,0.250000,2.500000,0.0,0.00000,1.000000,6.87500,0.0,0.5000,2.500,9.000000,7.750000
2,000062e9be78f3da274fec338e78f89d12000e781967f2...,68,7,27.0,1.025302e+07,61.0,0.0,0,0.0,0.000000,3.857143,0.5,0.00000,0.000000,12.12500,0.0,0.0000,2.500,29.593750,13.335938
3,00006ad44c0d33a2a526caf75514643639b7b7070dfffd...,57,3,7.0,4.104550e+06,55.0,0.0,0,0.0,0.000000,2.333333,0.0,0.00000,0.000000,5.25000,0.0,0.0000,3.000,9.000000,9.500000
4,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,22,3,3.0,9.135007e+05,20.0,0.0,0,0.0,0.000000,1.000000,0.0,0.00000,0.000000,0.50000,0.0,0.0000,1.000,42.500000,0.500000


In [88]:
df14a.to_csv(results_path + "/allfeatures_14.csv", index=False)